# Palpx Task

## Step 1 : Download Necessary Libraries

In [31]:
## Run this cell to download nltk. 
## If this library exists on your pc, then the cell returns "Requirement already satisfied"
!pip install nltk

## Make sure you download punkt
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
## Run this cell to download BeautifulSoup. 
## If this library exists on your pc, then the cell returns "Requirement already satisfied"
!pip install beautifulsoup4

In [32]:
## Run this cell to download Fuzzy Wuzzy. 
## If this library exists on your pc, then the cell returns "Requirement already satisfied"
!pip install fuzzywuzzy

In [34]:
## Run this cell to download Regular Expressions. 
## If this library exists on your pc, then the cell returns "Requirement already satisfied"
!pip install regex

In [35]:
## Run this cell to download tqdm. 
## If this library exists on your pc, then the cell returns "Requirement already satisfied" 
!pip install tqdm

In [36]:
## Run this cell to download Spacy. 
## If this library exists on your pc, then the cell returns "Requirement already satisfied" 
!pip install -U spacy

Requirement already up-to-date: spacy in /anaconda2/envs/py3k/lib/python3.7/site-packages (2.1.4)


In [37]:
## Run this cell to download Model for English on spacy. 
## If this package exists on your pc, then the cell returns "Requirement already satisfied"
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/anaconda2/envs/py3k/lib/python3.7/site-packages/en_core_web_sm -->
/anaconda2/envs/py3k/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


### Import all the Libraries

In [1]:
## Import all the necessary Libraries required to run this program
import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup
import re

from nltk import tokenize

from fuzzywuzzy import fuzz

from tqdm import tqdm_notebook as tqdm

import operator

import spacy
from spacy import displacy

## Step 2 : Input the Article link

In [2]:
## The link for the Wikipedia page I webscraped - https://en.wikipedia.org/wiki/R2-D2 
## page_url = "https://en.wikipedia.org/wiki/R2-D2"

page_url = input("Enter from where you need to Extract the Article! : ")

Enter from where you need to Extract the Article! : https://en.wikipedia.org/wiki/R2-D2


In [3]:
## Print the HTTP status code. If it returns 200,then the request has succeeded.

page = requests.get(page_url)
print(page.status_code)


200


In [4]:
## Let us extract all the Text Data - to understand better Refer README file
soup = BeautifulSoup(page.text)
text_file = soup.find_all('p')

In [5]:
## Print an example to understand what we have got
text_file[0:3]

[<p class="mw-empty-elt">
 </p>,
 <p><b>R2-D2</b> (<span class="nowrap"><span class="IPA nopopups noexcerpt"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˌ/: secondary stress follows">ˌ</span><span title="/ɑːr/: 'ar' in 'far'">ɑːr</span><span title="'t' in 'tie'">t</span><span title="/uː/: 'oo' in 'goose'">uː</span><span title="/ˈ/: primary stress follows">ˈ</span><span title="'d' in 'dye'">d</span><span title="/iː/: 'ee' in 'fleece'">iː</span><span title="'t' in 'tie'">t</span><span title="/uː/: 'oo' in 'goose'">uː</span></span>/</a></span></span>), or <b>Artoo-Detoo</b>,<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> is a fictional robot character in the <a href="/wiki/Star_Wars" title="Star Wars"><i>Star Wars</i> franchise</a> created by <a href="/wiki/George_Lucas" title="George Lucas">George Lucas</a>. A small <a class="mw-redirect" href="/wiki/Astromech_droid" title="Astromech droid">as

In [48]:
## Let us remove all the HTML Tags to obtain a much Clearer Text
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', str(data))
    cleanr = re.compile(', ')
    cleantext = re.sub(cleanr, ',', str(cleantext))
    cleanr = re.compile('\'')
    cleantext = re.sub(cleanr, '', str(cleantext))
    cleanr = re.compile('\n,')
    cleantext = re.sub(cleanr, ' ', str(cleantext))
    
    return cleantext

In [49]:
data = striphtml(text_file)

In [50]:
## Print an example to understand the data at hand
print(data[0:3000])

[ R2-D2 (/ˌɑːrtuːˈdiːtuː/),or Artoo-Detoo,[1] is a fictional robot character in the Star Wars franchise created by George Lucas. A small astromech droid,R2-D2 is a major character and appears in nine out of the ten Star Wars films to date. Throughout the course of the films,R2 is a friend to C-3PO,Padmé Amidala,Anakin Skywalker,Leia Organa,Luke Skywalker,and Obi-Wan Kenobi in various points in the saga. English actor Kenny Baker played R2-D2 in all three original Star Wars films,and received billing credit for the character in the prequel trilogy,where Bakers role was reduced,as R2-D2 was portrayed mainly by radio controlled props and CGI models. In the sequel trilogy,Baker was credited as consultant for The Force Awakens; however,Jimmy Vee also co-performed the character in some scenes. Vee later took over the role beginning in The Last Jedi.[2] R2-D2s sounds and vocal effects were created by Ben Burtt. R2-D2 was designed in artwork by Ralph McQuarrie,co-developed by John Stears and b

In [9]:
## This function sent_tokenize helps identify sentences from the data
sentences = tokenize.sent_tokenize(data)

In [10]:
## Print the length, to understand the number of sentences contained in this particular article
len(sentences)

99

## Step 3 : Input Question to Obtain the Context

In [11]:
## Set the question variable to obtain the Context.
question = input("Enter any question related to the article: ")

Enter any question related to the article: When was R2-D2 inducted into the Robot Hall of Fame?


### Finding Context

In [12]:
## By searching through the article we can find where the context related to the Question is present
## TQDM prints the time taken to run this Loop. This visually aids to understand the time taken to run this cell.

scores = []
index = []

for i in tqdm(range(len(sentences))) :
    score = fuzz.partial_ratio(question, sentences[i])
    scores.append(score)
    index.append(i)
    
dictionary_scores = dict(zip(index, scores))

probable_answer_index = max(dictionary_scores.items(), key=operator.itemgetter(1))[0]
score = dictionary_scores[probable_answer_index]

In [13]:
context = []

for i in range(probable_answer_index-2, probable_answer_index+2):
    context.append(sentences[i])

In [14]:
print(context)

['[32] Baker himself has said he probably only appears in footage caught while shooting the previous two movies.', '[33] For The Force Awakens,producer Kathleen Kennedy hired two fans,Lee Towarsey and Oliver Steeples,to build new R2-D2 robots for the film,after being impressed by their working replicas that were brought to Star Wars Celebration Europe in 2013.', '[34] R2-D2 was inducted into the Robot Hall of Fame in 2003.', 'The Smithsonian Institution included R2-D2 in its list of 101 Objects that Made America.']


## Step 4 : Use the Context to obtain the possible Answer

In [15]:
## Lets use Spacy to find the Dependencies of each word in the sentence.
## Using the dependencies we can extract the a) Root word, b) Link between the subject and the verb c) Link between verb and Objects

imp_word = []

nlp = spacy.load("en")
doc = nlp(question)

for token in doc:
    if token.dep_ == 'ROOT':
        imp_word.append(token.text)
    elif token.dep_ == 'obj' or (token.dep_ == 'dobj') or (token.dep_ == 'iobj') or (token.dep_ == 'pobj'):
        imp_word.append(token.text)
    elif (token.dep_ == 'subj') or (token.dep_ == 'nsubj') or (token.dep_ == 'nsubjpass') or (token.dep_ == 'csubj') or (token.dep_ == 'csubjpass') :
        imp_word.append(token.text)
        
key_phrase = ' '.join(imp_word)

In [16]:
## Print the key phrase.
key_phrase

'inducted Hall Fame'

In [17]:
## Use the key phrase to obtain the answer from the context

for sentence in context :
    score_2 = fuzz.token_set_ratio(sentence, key_phrase)
    
    if score_2 >= 50:
        print(sentence)
        print("Score : {}".format(score))

[34] R2-D2 was inducted into the Robot Hall of Fame in 2003.
Score : 83


## Step 5 : Testing 

To understand the performance, I have tested it by changing the question & also tried using the Logic to find possible answers from other Wikipedia Article.

1. Question 1 :

Question - Who was credited as consultant for The Force Awakens ?

Answer - In the sequel trilogy,Baker was credited as consultant for The Force Awakens; however,Jimmy Vee also co-performed the character in some scenes.

Score : 93

2. Question 2 :

Question - Who appears in nine out of the ten Star Wars films ?

A small astromech droid,R2-D2 is a major character and appears in nine out of the ten Star Wars films to date.

Score : 92

Let's try it out with other Article from Wikipedia.
I have used https://en.wikipedia.org/wiki/Elon_Musk

3. Question 3 :

Question - Who is Elon Musk ?

[325] In December 2016,Robby in New Girl refers to saving a life,and when asked who,responds "Do you know who Elon Musk is?".

Score : 78

4. Question 4 :

Question - Who founded SpaceX ?

[14][15][16][17] In May 2002,Musk founded SpaceX,an aerospace manufacturer and space transport services company,of which he is CEO and lead designer.

Score : 90

5. Question 5 :

Question - Where did Elon Musk attend University ?

[43][44] At the age of 17,in 1989,Elon Musk moved to Canada to attend Queens University,avoiding mandatory service in the South African military.

Score : 64

He left in 1992 to study business and physics at the University of Pennsylvania,and graduated with an undergraduate degree in economics and stayed for a second bachelors degree in physics.

Score : 64

[45][46] After leaving Penn,Elon Musk headed to Stanford University in California to pursue a PhD in energy physics.

Score : 64

Well, this gives multiple answers with a same confidence score as its obvious that Elon Musk attended more than 1 university. A Clear question stating the name of the University should return a single sentence as the answer with a much better Score

In [39]:
def find_answer (sentences, question):
    
    scores = []
    index = []

    for i in tqdm(range(len(sentences))) :
        score = fuzz.partial_ratio(question, sentences[i])
        scores.append(score)
        index.append(i)

    dictionary_scores = dict(zip(index, scores))

    probable_answer_index = max(dictionary_scores.items(), key=operator.itemgetter(1))[0]
    score = dictionary_scores[probable_answer_index]
    
    context = []
    
    for i in range(probable_answer_index-2, probable_answer_index+2):
        context.append(sentences[i])
    
    imp_word = []
    
    nlp = spacy.load("en")
    doc = nlp(question)
    
    for token in doc:
        if token.dep_ == 'ROOT':
            imp_word.append(token.text)
        elif token.dep_ == 'obj' or (token.dep_ == 'dobj') or (token.dep_ == 'iobj') or (token.dep_ == 'pobj'):
            imp_word.append(token.text)
        elif (token.dep_ == 'subj') or (token.dep_ == 'nsubj') or (token.dep_ == 'nsubjpass') or (token.dep_ == 'csubj') or (token.dep_ == 'csubjpass') :
            imp_word.append(token.text)
    
    key_phrase = ' '.join(imp_word)
    
    
    for sentence in context :
        score_2 = fuzz.token_set_ratio(sentence, key_phrase)
        
        if score_2 >= 50:
            print(sentence)
            print("Score : {}".format(score))
            

def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', str(data))
    cleanr = re.compile(', ')
    cleantext = re.sub(cleanr, ',', str(cleantext))
    cleanr = re.compile('\'')
    cleantext = re.sub(cleanr, '', str(cleantext))
    cleanr = re.compile('\n,')
    cleantext = re.sub(cleanr, ' ', str(cleantext))
    
    return cleantext            
            
def obtain_sentences(page_url):
    page = requests.get(page_url)
    
    soup = BeautifulSoup(page.text)
    text_file = soup.find_all('p')
    
    data = striphtml(text_file)
    
    sentences = tokenize.sent_tokenize(data)
    
    return sentences

### Question 1 : Who was credited as consultant for The Force Awakens ?

In [40]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Who was credited as consultant for The Force Awakens ?



In the sequel trilogy,Baker was credited as consultant for The Force Awakens; however,Jimmy Vee also co-performed the character in some scenes.
Score : 93


### Question 2 : Who appears in nine out of the ten Star Wars films ?

In [41]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Who appears in nine out of the ten Star Wars films ?



A small astromech droid,R2-D2 is a major character and appears in nine out of the ten Star Wars films to date.
Score : 92


### Question 3 : Who is Elon Musk ?

In [51]:
page_url = input("Enter the link of the page, from where you need to Extract the Article! : ")
sentences = obtain_sentences(page_url)

Enter the link of the page, from where you need to Extract the Article! : https://en.wikipedia.org/wiki/Elon_Musk


In [52]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Who is Elon Musk ?



[325] In December 2016,Robby in New Girl refers to saving a life,and when asked who,responds "Do you know who Elon Musk is?".
Score : 78


### Question 4 : Who founded SpaceX ? An aerospace manufacturer and space transport services company.

In [55]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Who founded SpaceX ? An aerospace manufacturer and space transport services company.



[14][15][16][17] In May 2002,Musk founded SpaceX,an aerospace manufacturer and space transport services company,of which he is CEO and lead designer.
Score : 90


### Question 5 : Where did Elon Musk attend University ?

In [56]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Where did Elon Musk attend University ?



[43][44] At the age of 17,in 1989,Elon Musk moved to Canada to attend Queens University,avoiding mandatory service in the South African military.
Score : 64
He left in 1992 to study business and physics at the University of Pennsylvania,and graduated with an undergraduate degree in economics and stayed for a second bachelors degree in physics.
Score : 64
[45][46] After leaving Penn,Elon Musk headed to Stanford University in California to pursue a PhD in energy physics.
Score : 64
